# Experiments

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="/home/jai/study_material/sem7/llm/code_work/Jaideep_Singh_2210110675_MAT496/.env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [6]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [9]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "jai_rag_dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-07998a8e' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=8d82225c-18f8-458c-8746-cdf6c4791305




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,2.029331,0200895c-35f1-4959-9a81-07bf27d54603,8705ad4e-2f52-4b4b-a7f2-750ab976e60d
1,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.326070,26fa896e-e839-438b-ad81-2f72322fe0aa,691e45a5-23d0-42e9-8e11-fc381a275691
2,Does LangSmith support offline evaluation?,LangSmith does not currently support offline e...,None,"Yes, LangSmith supports offline evaluation thr...",0,2.559844,2a36a43a-78e7-4464-9bdc-4939606c1a3e,049b03fe-85ca-4f53-9213-771de34cc89c
3,Can LangSmith be used for finetuning and model...,"LangSmith is primarily focused on monitoring, ...",None,"Yes, LangSmith can be used for fine-tuning and...",0,1.631252,5883498e-1b3f-45fb-9134-f76998a89e00,1b35315d-46c7-496f-9a2a-689d3ec919c3
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for the creat...,None,LangSmith is a platform designed for the devel...,1,2.250643,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,18ef59e1-b6aa-4a81-ae4a-1a0b6f0d89b3
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.427757,90b193d4-2d97-468b-807b-bde01655cd8c,1e90f310-c746-46a9-9086-9a69d3985658
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.841979,981cb9c9-444e-4c86-bf29-6169a839c666,2ed830a8-161f-4254-9294-84c47d3d0e88
7,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.637006,d5e778a9-84d3-44df-9302-711859240dd4,d2c6247c-d226-4fa2-b30d-223698ed73e5
8,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can i...",None,You can pass metadata with the @traceable deco...,0,2.815969,da756d22-16d0-4a36-a148-ac517c66591a,5f80b104-07bc-463f-ae9d-9f3fc19d9c5c
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.906713,de2b7e5e-711b-47f0-a35a-d9275dfeca04,08238842-b046-4da7-8dd1-10760ae9233e


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [8]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-e209a69f' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=0773dbbc-0b30-4f60-8741-30503db6cd66




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation ...",None,"Yes, LangSmith supports online evaluation as a...",0,1.831199,0200895c-35f1-4959-9a81-07bf27d54603,5e51ce36-d851-4e76-91b6-940160536eb2
1,What testing capabilities does LangSmith have?,LangSmith in LangChain provides testing capabi...,None,LangSmith offers capabilities for creating dat...,1,2.247141,26fa896e-e839-438b-ad81-2f72322fe0aa,c6ddcd38-4ceb-4f46-9fda-beeb3bd52dfb
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.455726,2a36a43a-78e7-4464-9bdc-4939606c1a3e,72787fed-cfcb-45d9-9b95-834bf1a6ed61
3,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.530675,5883498e-1b3f-45fb-9134-f76998a89e00,0e424b5b-c4cd-45cb-b48c-d192452f5d33
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.839091,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,00bcd35c-4131-4b6b-b5b2-e7b7f02a1146
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.683367,90b193d4-2d97-468b-807b-bde01655cd8c,04ae6893-4bda-4819-929b-a20609e0d9af
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,1.687799,981cb9c9-444e-4c86-bf29-6169a839c666,40c0d3b4-23c9-4d37-9fbd-e5bfbb3402ba
7,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.837775,d5e778a9-84d3-44df-9302-711859240dd4,0bb67ea1-5e58-4cda-a63b-704cfc357078
8,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can u...",None,You can pass metadata with the @traceable deco...,1,2.337953,da756d22-16d0-4a36-a148-ac517c66591a,83aba2ee-dae4-4a04-95a3-124f3730b298
9,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.941540,de2b7e5e-711b-47f0-a35a-d9275dfeca04,cc74ef2b-9647-409c-80e6-ae5c8f57fdd6


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [12]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="latest"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-5bf9c380' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=79e7fa01-7e17-4e0b-8d68-3c56cb272489




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,2.658626,0200895c-35f1-4959-9a81-07bf27d54603,201cf5e8-fb9d-47d2-a51f-8b63c8588d3a
1,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,2.039853,26fa896e-e839-438b-ad81-2f72322fe0aa,84344b5a-3a92-456b-b3d4-4348c1242bfb
2,Does LangSmith support offline evaluation?,LangSmith does not support offline evaluation ...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.647883,2a36a43a-78e7-4464-9bdc-4939606c1a3e,da65de8f-b792-456b-8ef7-a832a62e74e7
3,Can LangSmith be used for finetuning and model...,"LangSmith is primarily focused on monitoring, ...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.751578,5883498e-1b3f-45fb-9134-f76998a89e00,702e78bb-be77-490b-bc70-751ef3dc8c75
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.530809,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,6eacc5d4-ab2d-4f07-96b1-3033cc568147
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.421660,90b193d4-2d97-468b-807b-bde01655cd8c,dc51c164-6ad6-478a-8e2f-a44308682b31
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,1.743936,981cb9c9-444e-4c86-bf29-6169a839c666,c397937a-b609-4481-9feb-1dacfeb869f0
7,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.745989,d5e778a9-84d3-44df-9302-711859240dd4,5acb189f-7d3a-46df-b9a2-c1e04497c7d0
8,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,1.416060,da756d22-16d0-4a36-a148-ac517c66591a,b84631ae-475d-461f-bf4a-2086b846b711
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.836277,de2b7e5e-711b-47f0-a35a-d9275dfeca04,8a5c7e90-7b5c-4e15-ad6b-d3ebde0d6675


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [16]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Jai's Split"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-e320f563' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=73ac5aee-8a78-4c11-8b84-a32e62eeffa9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"LangSmith does support online evaluation, prov...",None,"Yes, LangSmith supports online evaluation as a...",0,1.934270,0200895c-35f1-4959-9a81-07bf27d54603,eafa34a0-9793-426a-a698-e1687bf94804
1,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the capability t...,None,LangSmith offers capabilities for creating dat...,1,1.635461,26fa896e-e839-438b-ad81-2f72322fe0aa,e90abdf5-f267-4f4a-8e11-dffa9072c1f6
2,Does LangSmith support offline evaluation?,LangSmith does not currently support offline e...,None,"Yes, LangSmith supports offline evaluation thr...",0,2.448051,2a36a43a-78e7-4464-9bdc-4939606c1a3e,14a29236-593c-4ebf-b898-470c6e04f4d9
3,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on monitoring a...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.246352,5883498e-1b3f-45fb-9134-f76998a89e00,ee39ce65-3e30-4db2-bae0-abeb01fd3b59
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.945866,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,573b89d6-6cf7-4555-9335-2e9181fabb54
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,2.349275,90b193d4-2d97-468b-807b-bde01655cd8c,e431d27a-cf89-4b22-b233-e5723cccd22f
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.779799,981cb9c9-444e-4c86-bf29-6169a839c666,fe21a70f-076b-46f3-a33e-125c38350182
7,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.409551,d5e778a9-84d3-44df-9302-711859240dd4,1ce72b03-4e0d-46a6-bfb5-7488aabfe1fb
8,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you can...",None,You can pass metadata with the @traceable deco...,1,1.634781,da756d22-16d0-4a36-a148-ac517c66591a,e0e0b771-67f9-4278-a768-23570f0d0ed0
9,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.938050,de2b7e5e-711b-47f0-a35a-d9275dfeca04,d7aef5a4-40a7-4742-99ac-ef06bc1340b4


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [17]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "2a36a43a-78e7-4464-9bdc-4939606c1a3e",
            "6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-1ef29025' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=8dc1fea0-c8f7-425d-b659-aea9212849f4




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,"LangSmith supports online evaluations, providi...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.939302,2a36a43a-78e7-4464-9bdc-4939606c1a3e,f1918ca9-9a18-4df6-8ddf-f90f50920529
1,What is LangSmith used for in three sentences?,LangSmith is used as a platform for building p...,None,LangSmith is a platform designed for the devel...,1,1.733054,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,0fb9966b-1036-41f4-9eb6-b0c5fe87d665


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [18]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-c67a2671' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=667117ff-abfc-47a7-a2b3-7d8eab011c05




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,2.344372,0200895c-35f1-4959-9a81-07bf27d54603,7e2db176-f45e-4690-9a06-b11cc3a1bd7d
1,What testing capabilities does LangSmith have?,LangSmith offers the ability to run multiple e...,None,LangSmith offers capabilities for creating dat...,1,1.845208,26fa896e-e839-438b-ad81-2f72322fe0aa,6d168429-117a-4359-bbcd-cb7f0173e0f3
2,Does LangSmith support offline evaluation?,LangSmith supports online evaluations for real...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.746356,2a36a43a-78e7-4464-9bdc-4939606c1a3e,28e00996-2cde-4022-8edb-e78380289e1e
3,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",0,1.725017,5883498e-1b3f-45fb-9134-f76998a89e00,15632a0f-7ece-4df5-8501-510dcd0dd445
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.430006,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,c1e150c6-d1cb-4515-a752-5940df8581ad
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,2.144479,90b193d4-2d97-468b-807b-bde01655cd8c,894e6682-cbd4-4106-9167-8aca47ccc0cb
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,1,2.146084,981cb9c9-444e-4c86-bf29-6169a839c666,dbe50d88-7b1d-45dc-bc08-a4c148010c45
7,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,2.041505,d5e778a9-84d3-44df-9302-711859240dd4,2570a07e-8913-42a9-aa80-24eba743bd8f
8,How do I pass metadata in with @traceable?,"To pass metadata with **@traceable**, you can ...",None,You can pass metadata with the @traceable deco...,1,1.940624,da756d22-16d0-4a36-a148-ac517c66591a,69434784-37f6-40a3-9bd7-3c4346f158c6
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.638895,de2b7e5e-711b-47f0-a35a-d9275dfeca04,b6938bf6-ea5d-4824-99ea-49aa49edab71


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [19]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-ab9689b9' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=f0d7cf92-edd5-4c4b-9768-9c400134921f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.824485,26fa896e-e839-438b-ad81-2f72322fe0aa,7fffe6a8-cd8e-4696-b123-8d8d81d29f32
1,Does LangSmith support online evaluation?,LangSmith supports online evaluation through i...,None,"Yes, LangSmith supports online evaluation as a...",0,2.346452,0200895c-35f1-4959-9a81-07bf27d54603,2c37ee3a-ea56-40de-85fd-b1e0e19ddc22
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.431498,2a36a43a-78e7-4464-9bdc-4939606c1a3e,f83961fd-45a7-4ac0-ba61-8532ee811a69
3,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.436327,5883498e-1b3f-45fb-9134-f76998a89e00,ee48d096-01a1-4c5a-b5af-9a537590c70d
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.643291,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,30e4ee62-7226-4e7c-a1af-ae63b893da49
5,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.090190,90b193d4-2d97-468b-807b-bde01655cd8c,7138f1ef-1cc7-40b1-b607-7930e5e6e863
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.638492,981cb9c9-444e-4c86-bf29-6169a839c666,064729b4-40e8-4a22-95f8-64912d395753
7,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.036218,d5e778a9-84d3-44df-9302-711859240dd4,e416f41c-2d37-4837-bd5a-ed8a43a66b73
8,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.637995,de2b7e5e-711b-47f0-a35a-d9275dfeca04,2dce87b5-bcad-4ca3-994f-d03bf07127ec
9,How can I integrate LangSmith with existing ap...,To integrate LangSmith with existing applicati...,None,You can integrate LangSmith with existing appl...,0,1.841650,11de7bae-a0b3-4458-87ab-b96e26851016,463eebe0-f6d6-49e2-85ba-989e9f4f43bc


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [20]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": 'gpt-4o-mini'
    }
)

View the evaluation results for experiment: 'metadata added-e7ed159c' at:
https://smith.langchain.com/o/2c92d665-c688-4c16-a10e-362fe0bb7d80/datasets/ad628c31-7b6d-4846-9464-3b7bbc6f6cfb/compare?selectedSessions=941d88ef-752c-4f26-8ef6-436048a15e52




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangChain provides off-the-shelf evaluato...",None,"Yes, LangSmith supports online evaluation as a...",0,5.109412,0200895c-35f1-4959-9a81-07bf27d54603,bc9f8f7f-a977-4d4f-b448-35e23e79f8b8
1,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the capability t...,None,LangSmith offers capabilities for creating dat...,1,1.530966,26fa896e-e839-438b-ad81-2f72322fe0aa,256aa3bc-820a-414c-bff8-f9672be6eb26
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.375520,2a36a43a-78e7-4464-9bdc-4939606c1a3e,628797dc-a93c-4d88-97f7-a61dc0b585db
3,Can LangSmith be used for finetuning and model...,"LangSmith is primarily used for monitoring, ev...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.632981,5883498e-1b3f-45fb-9134-f76998a89e00,a5a01a7e-a506-463c-b5d4-c1e7a85edfa2
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,0,1.788308,6cd765a4-ed93-4d2a-86a8-a5f2d1e949a7,ed51e469-1e25-457d-ad6a-add51ae7078c
5,Can LangSmith be used to evaluate agents?,LangSmith can be used to evaluate agents. It i...,None,"Yes, LangSmith can be used to evaluate agents....",1,1.441100,90b193d4-2d97-468b-807b-bde01655cd8c,86029668-972c-48fa-9a28-e720de7ceb4d
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,1.860978,981cb9c9-444e-4c86-bf29-6169a839c666,ebabc6ff-885c-4277-8a93-8c87aa9e71b3
7,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.901818,d5e778a9-84d3-44df-9302-711859240dd4,17b2203e-fde7-4f66-8ede-45d6ed22509a
8,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you can...",None,You can pass metadata with the @traceable deco...,1,1.876530,da756d22-16d0-4a36-a148-ac517c66591a,08bf3013-06ff-4280-9be8-1f14d1dfd53c
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,2.611825,de2b7e5e-711b-47f0-a35a-d9275dfeca04,dcf2c7a2-4e3a-4431-94f2-af3b5b6011ae
